In [4]:
#Package versions required for succesful model unpack inside the KFserving container

!pip install --upgrade scikit-learn==0.20.3 joblib==0.17.0 numpy==1.19.4

     |████████████████████████████████| 5.4 MB 8.4 MB/s eta 0:00:01
Requirement already up-to-date: joblib==0.17.0 in /opt/conda/lib/python3.7/site-packages (0.17.0)
     |████████████████████████████████| 14.5 MB 74.5 MB/s eta 0:00:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.18.5
    Uninstalling numpy-1.18.5:
      Successfully uninstalled numpy-1.18.5
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.23.2
    Uninstalling scikit-learn-0.23.2:
      Successfully uninstalled scikit-learn-0.23.2
ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

tensorflow-cpu 2.3.1 requires numpy<1.19.0,>=1.16.0, but you'll have numpy 1.19.4 which is incompatible.


In [ ]:
import pickle
#imported in randomforest
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
import joblib

#Read Data
data_path=('Data.csv')
df_churn1 = pd.read_csv(data_path)

#Data Encoding for Random Forest Model
def convert_data(df_churn):
    empty_cols=['customerID', 'gender', 'SeniorCitizen', 'Partner', 'Dependents',
           'tenure', 'PhoneService', 'MultipleLines', 'InternetService',
           'OnlineSecurity', 'OnlineBackup', 'DeviceProtection','TechSupport',
           'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling',
           'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'Churn']
    for i in empty_cols:
        df_churn[i]=df_churn[i].replace(" ",np.nan)

    df_churn.drop('cluster_number', axis=1, inplace=True)
    df_churn.drop('customerID', axis=1, inplace=True)
    df_churn=df_churn.dropna()
    
    #Binary Encoding
    binary_cols=['Partner','Dependents','PhoneService','PaperlessBilling']

    for i in binary_cols:
        df_churn[i]=df_churn[i].replace({"Yes":1,"No":0})

    #Encoding column 'gender'
    df_churn['gender']=df_churn['gender'].replace({"Male":1,"Female":0})

    #Categorical Encoding
    category_cols=['PaymentMethod','MultipleLines','InternetService','OnlineSecurity','OnlineBackup','DeviceProtection','TechSupport','StreamingTV','StreamingMovies','Contract']

    for cc in category_cols:
        dummies = pd.get_dummies(df_churn[cc], drop_first=False)
        dummies = dummies.add_prefix("{}#".format(cc))
        df_churn.drop(cc, axis=1, inplace=True)
        df_churn = df_churn.join(dummies)
    df_churn['Churn']=df_churn['Churn'].replace({"Yes":1,"No":0})
    return df_churn


df_churn2 = convert_data(df_churn1)
df_churn2.to_csv('Data_modified.csv', index=False)
print('Converting done!')

In [ ]:
#random forest code

print('process started')

data_path1 = ('Data_modified.csv')
df = pd.read_csv(data_path1)

X = df.loc[:, df.columns != 'Churn']
y = df.loc[:, df.columns == 'Churn']

rfc_best=RandomForestClassifier(random_state=42, max_features='auto', n_estimators= 50, max_depth=8,
                                criterion='gini')

rfc_best.fit(X, y)

filename = 'finalized_model.sav'
pickle.dump(rfc_best, open(filename, 'wb'))
joblib.dump(rfc_best, open('model.joblib', 'wb'))
print('dump complete')
